In [1]:
path = 'tswiftlyrics.txt'

In [2]:
import tensorflow as tf

2023-01-05 08:36:20.214759: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
text = open(path, 'rb').read().decode(encoding='utf-8').strip("\n")
print(text[:250])

[Verse 1]
I walked through the door with you, the air was cold
But somethin' 'bout it felt like home somehow
And I left my scarf there at your sister's house
And you've still got it in your drawer, even now

[Verse 2]
Oh, your sweet disposition and m


In [4]:
print(f'Length of text: {len(text)} characters')

Length of text: 706287 characters


In [5]:
chars = sorted(set(text))
print(str(len(chars)) + ' unique characters')

96 unique characters


In [6]:
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 'é',
 'í',
 'ó',
 'е',
 '\u2005',
 '\u200b',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '…',
 '\u205f']

In [7]:
ex = ['shreeya', 'hello']
c = tf.strings.unicode_split(ex, input_encoding='UTF-8')
c

2023-01-05 08:36:24.638836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.RaggedTensor [[b's', b'h', b'r', b'e', b'e', b'y', b'a'],
 [b'h', b'e', b'l', b'l', b'o']]>

In [8]:
ch_to_id = tf.keras.layers.StringLookup(vocabulary=list(chars), mask_token=None)

In [9]:
ch_to_id(c)

<tf.RaggedTensor [[74, 63, 73, 60, 60, 80, 56], [63, 60, 67, 67, 70]]>

In [10]:
id_to_ch = tf.keras.layers.StringLookup(vocabulary=list(chars), invert=True, mask_token=None)

In [11]:
id_to_ch(ch_to_id(c))

<tf.RaggedTensor [[b's', b'h', b'r', b'e', b'e', b'y', b'a'],
 [b'h', b'e', b'l', b'l', b'o']]>

In [12]:
def id_to_text(ids):
    return tf.strings.reduce_join(id_to_ch(ids), axis=-1)

In [13]:
id_to_text(ch_to_id(c))

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'shreeya', b'hello'], dtype=object)>

In [14]:
all_id = ch_to_id(tf.strings.unicode_split(text, input_encoding='UTF-8'))
all_id

<tf.Tensor: shape=(706287,), dtype=int64, numpy=array([54, 49, 60, ..., 56, 75, 60])>

In [15]:
id_dataset = tf.data.Dataset.from_tensor_slices(all_id)
for ids in id_dataset.take(10):
    print(id_to_ch(ids).numpy().decode('utf-8'))

[
V
e
r
s
e
 
1
]




In [16]:
seq_len = 100

In [17]:
sequences = id_dataset.batch(seq_len, drop_remainder=True)

for seq in sequences.take(1):
    print(id_to_ch(seq))

tf.Tensor(
[b'[' b'V' b'e' b'r' b's' b'e' b' ' b'1' b']' b'\n' b'I' b' ' b'w' b'a'
 b'l' b'k' b'e' b'd' b' ' b't' b'h' b'r' b'o' b'u' b'g' b'h' b' ' b't'
 b'h' b'e' b' ' b'd' b'o' b'o' b'r' b' ' b'w' b'i' b't' b'h' b' ' b'y'
 b'o' b'u' b',' b' ' b't' b'h' b'e' b' ' b'a' b'i' b'r' b' ' b'w' b'a'
 b's' b' ' b'c' b'o' b'l' b'd' b'\n' b'B' b'u' b't' b' ' b's' b'o' b'm'
 b'e' b't' b'h' b'i' b'n' b"'" b' ' b"'" b'b' b'o' b'u' b't' b' ' b'i'
 b't' b' ' b'f' b'e' b'l' b't' b' ' b'l' b'i' b'k' b'e' b' ' b'h' b'o'
 b'm' b'e'], shape=(100,), dtype=string)


In [18]:
for seq in sequences.take(5):
    print(id_to_text(seq))

tf.Tensor(b"[Verse 1]\nI walked through the door with you, the air was cold\nBut somethin' 'bout it felt like home", shape=(), dtype=string)
tf.Tensor(b" somehow\nAnd I left my scarf there at your sister's house\nAnd you've still got it in your drawer, ev", shape=(), dtype=string)
tf.Tensor(b"en now\n\n[Verse 2]\nOh, your sweet disposition and my wide-eyed gaze\nWe're singin' in the car, getting", shape=(), dtype=string)
tf.Tensor(b" lost upstate\nAutumn leaves fallin' down like pieces into place\nAnd I can picture it after all these", shape=(), dtype=string)
tf.Tensor(b" days\n\n[Pre-Chorus]\nAnd I know it's long gone and\nThat magic's not here no more\nAnd I might be okay,", shape=(), dtype=string)


In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Shreeya"))

(['S', 'h', 'r', 'e', 'e', 'y'], ['h', 'r', 'e', 'e', 'y', 'a'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", id_to_text(input_example).numpy())
    print("Target:", id_to_text(target_example).numpy())

Input : b"[Verse 1]\nI walked through the door with you, the air was cold\nBut somethin' 'bout it felt like hom"
Target: b"Verse 1]\nI walked through the door with you, the air was cold\nBut somethin' 'bout it felt like home"


In [23]:
dataset = dataset.shuffle(1000).batch(64, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [24]:
vocab_size = len(ch_to_id.get_vocabulary())

In [25]:
vocab_size

97

In [26]:
len(chars)

96

In [27]:
embedding_dim = 256
rnn_units = 1024

In [28]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [29]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [105]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 99, 97) # (batch_size, sequence_length, vocab_size)


In [96]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24832     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  99425     
                                                                 
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [106]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([12, 18, 84, 45, 31, 89, 67,  4, 44, 58, 17, 85, 11, 67, 57, 43, 23,
       12,  3, 27, 22,  0, 74, 31, 16, 92, 80, 54, 27, 66, 62, 65, 27, 16,
       20, 92,  6, 38, 54, 75, 86, 24,  9,  9, 11, 44, 12, 45, 76, 78, 67,
       80, 61, 90, 16, 17, 85, 48, 59, 19, 57,  4, 80, 68, 86, 67, 26, 75,
       70, 75, 14, 28, 53, 92, 44, 71, 11, 15, 25, 78, 88, 28, 13, 39, 74,
       43,  1, 92, 16, 48, 68, 47, 28, 27,  1, 93,  5, 10, 35])

In [107]:
print("Input:\n", id_to_text(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", id_to_text(sampled_indices).numpy())

Input:
 b" like bang, we tried to forget it, but we just couldn't\nAnd I bury hatchets, but I keep maps of whe"

Next Char Predictions:
 b'-3\xc3\xadRD\xe2\x80\x93l"Qc2\xc3\xb3,lbP8-!?7[UNK]sD1\xe2\x80\x99y[?kgj?15\xe2\x80\x99\'K[t\xd0\xb59**,Q-Ruwlyf\xe2\x80\x9412\xc3\xb3Ud4b"ym\xd0\xb5l;tot/AZ\xe2\x80\x99Qp,0:w\xe2\x80\x8bA.LsP\n\xe2\x80\x991UmTA?\n\xe2\x80\x9c&+H'


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [109]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 99, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.575602, shape=(), dtype=float32)


In [110]:
tf.exp(example_batch_mean_loss).numpy()

97.08647

In [111]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [112]:
model.compile(optimizer='adam', loss=loss)

In [113]:
history = model.fit(dataset, epochs=20, callbacks=[checkpoint_callback])

Epoch 1/20
110/110 [==============================] - 161s 1s/step - loss: 2.9718
Epoch 2/20
110/110 [==============================] - 168s 2s/step - loss: 2.0667
Epoch 3/20
110/110 [==============================] - 174s 2s/step - loss: 1.7415
Epoch 4/20
110/110 [==============================] - 180s 2s/step - loss: 1.5267
Epoch 5/20
110/110 [==============================] - 165s 1s/step - loss: 1.3733
Epoch 6/20
110/110 [==============================] - 167s 2s/step - loss: 1.2536
Epoch 7/20
110/110 [==============================] - 176s 2s/step - loss: 1.1519
Epoch 8/20
110/110 [==============================] - 169s 2s/step - loss: 1.0586
Epoch 9/20
110/110 [==============================] - 172s 2s/step - loss: 0.9709
Epoch 10/20
110/110 [==============================] - 172s 2s/step - loss: 0.8866
Epoch 11/20
110/110 [==============================] - 176s 2s/step - loss: 0.8040
Epoch 12/20
110/110 [==============================] - 169s 2s/step - loss: 0.7225
Epoch 13/20
1

In [45]:
class OneStep(tf.keras.Model):
    def __init__(self, model, id_to_ch, ch_to_id, temperature=1.14):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.id_to_ch = id_to_ch
        self.ch_to_id = ch_to_id

    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ch_to_id(input_chars).to_tensor()

        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature

        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        predicted_chars = self.id_to_ch(predicted_ids)

        return predicted_chars, states

In [31]:
one_step_model = OneStep(model, id_to_ch, ch_to_id)

In [46]:
states = None
next_char = tf.constant(['['])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'))

NameError: name 'one_step_model' is not defined

In [46]:
model_load = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [47]:
model_load.load_weights('./training_checkpoints/ckpt_20')

In [48]:
one_step_model_loaded = OneStep(model_load, id_to_ch, ch_to_id)

In [51]:
states = None
next_char = tf.constant(['['])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model_loaded.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'))

[Chorus]
Didn't shake it off
I, I, I shake I wouldn't be exavitytice
'Cause you weren undrouth (Yeah, oh, yeah)
There you ever heard out up with you
Smile agains in your very first den and gon't go fie

[Chorus]
So it's gonna give you when you're sleeping
Keep your ey-eyes open

[Verse 2]
There's the fight of our live out dees, but never be somewhere, we're going down ‘roude and Cried

[Chorus]
"Come on an onvenseat of what I ever)
Screaming, "I'm in love with you"
Wait there in the pourin' rain, coming back forever, everything is wrong
It rains when you're here and it rains when you're gonna (I know place)
"Oh my God, look at that fall back on you)
Let my slil my love as, umberies and falling up now
But if I was a month on how tonight
All you drive me crazy chancing, sunny, Surfrife

[Pre-Chorus]
And it would be a flawe them
Every tim from the balcory
Even though the whole tcrow
Know you better than that
Huy you don’t know what you middled man
I got myself it in a new harder on a twin